# [RFCX] Sound Event Detection

Based on the work from [@hidehisaarai1213](https://www.kaggle.com/hidehisaarai1213) and [@gopidurgaprasad](https://www.kaggle.com/gopidurgaprasad)

Rainforest Connection (RFCX) Species Audio Detection is a Kaggle competition where the goal is to classify sounds of birds and other animals living in the rainforest. 

It's a sound tagging competition: for each test recording we have to predict the probability that each species is present in it. The goal is to maximize the label-weighted label-ranking average precision (LWLRAP) on the test set.

This notebook shows how a [Sound Event Detection](https://arxiv.org/abs/1912.10211) model can be used to do the task. After computing the mean prediction over all of the 5 folds one can achieve a LWLRAP score as high as 0.893. 

### Install packages

In [ ]:
!pip -q install --upgrade pip
!pip -q install audiomentations
!pip -q install timm
!pip -q install torchlibrosa

### Import libraries

In [ ]:
import audiomentations as aa
from functools import partial
import numpy as np
import os
import pandas as pd
import random
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import soundfile as sf
import time
from timm.models.efficientnet import tf_efficientnet_b0_ns
from timm.models.resnest import resnest50d
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchlibrosa.stft import Spectrogram, LogmelFilterBank
from torchlibrosa.augmentation import SpecAugmentation
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup

### Arguments

In [ ]:
class args:
    
    batch_size = 16
    debug = False
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    early_stop = 15
    epochs = 50
    epoch_scheduler = False
    folds = 5
    lr = 1e-3
    model_name = "SED_EFFNETB0"
    model_param = {
    'encoder': 'tf_efficientnet_b0_ns',
    'sample_rate': 48000,
    'window_size': 512,
    'hop_size': 512,
    'mel_bins': 384,
    'fmin': 0,
    'fmax': 24000,
    'classes_num': 24
    }
    num_workers = 4
    output_dir = "weights"
    pretrain_weights = None
    seed = 2021
    start_epoch = 0
    step_scheduler = True
    sub_csv = "../input/rfcx-species-audio-detection/sample_submission.csv"
    test_data_path = "../input/rfcx-species-audio-detection/test"
    train_csv = "train_folds.csv"
    train_data_path = "../input/rfcx-species-audio-detection/train"   

### Load data

In [ ]:
train_tp_df = pd.read_csv("../input/rfcx-species-audio-detection/train_tp.csv").sort_values("recording_id")
sub_df = pd.read_csv("../input/rfcx-species-audio-detection/sample_submission.csv")

train_gby_df = train_tp_df.groupby("recording_id")[["species_id"]].first().reset_index()
train_gby_df = train_gby_df.sample(frac=1, random_state=args.seed).reset_index(drop=True)
train_gby_df.loc[:, 'kfold'] = -1

X = train_gby_df["recording_id"].values
y = train_gby_df["species_id"].values

kfold = StratifiedKFold(n_splits=args.folds)
for fold, (t_idx, v_idx) in enumerate(kfold.split(X, y)):
    train_gby_df.loc[v_idx, "kfold"] = fold

train_tp_df = train_tp_df.merge(train_gby_df[['recording_id', 'kfold']], on="recording_id", how="left")
train_tp_df.to_csv("train_folds.csv", index=False)

### Define model architecture

In [ ]:
def init_layer(layer):
    nn.init.xavier_uniform_(layer.weight)
    if hasattr(layer, "bias"):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)


def init_bn(bn):
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.0)


def init_weights(model):
    classname = model.__class__.__name__
    if classname.find("Conv2d") != -1:
        nn.init.xavier_uniform_(model.weight, gain=np.sqrt(2))
        model.bias.data.fill_(0)
    elif classname.find("BatchNorm") != -1:
        model.weight.data.normal_(1.0, 0.02)
        model.bias.data.fill_(0)
    elif classname.find("GRU") != -1:
        for weight in model.parameters():
            if len(weight.size()) > 1:
                nn.init.orghogonal_(weight.data)
    elif classname.find("Linear") != -1:
        model.weight.data.normal_(0, 0.01)
        model.bias.data.zero_()


def do_mixup(x: torch.Tensor, mixup_lambda: torch.Tensor):
    """Mixup x of even indexes (0, 2, 4, ...) with x of odd indexes
    (1, 3, 5, ...).
    Args:
      x: (batch_size * 2, ...)
      mixup_lambda: (batch_size * 2,)
    Returns:
      out: (batch_size, ...)
    """
    out = (x[0::2].transpose(0, -1) * mixup_lambda[0::2] +
           x[1::2].transpose(0, -1) * mixup_lambda[1::2]).transpose(0, -1)
    return out


class Mixup(object):
    def __init__(self, mixup_alpha, random_seed=args.seed):
        """Mixup coefficient generator"""
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = []
        for n in range(0, batch_size, 2):
            lam = self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            mixup_lambdas.append(lam)
            mixup_lambdas.append(1. - lam)
            
        return torch.from_numpy(np.array(mixup_lambdas, dtype=np.float32))


def interpolate(x: torch.Tensor, ratio: int):
    """Interpolate data in time domain. This is used to compensate the
    resolution reduction in downsampling of a CNN.
    Args:
      x: (batch_size, time_steps, classes_num)
      ratio: int, ratio to interpolate
    Returns:
      upsampled: (batch_size, time_steps * ratio, classes_num)
    """
    (batch_size, time_steps, classes_num) = x.shape
    upsampled = x[:, :, None, :].repeat(1, 1, ratio, 1)
    upsampled = upsampled.reshape(batch_size, time_steps * ratio, classes_num)
    return upsampled


def pad_framewise_output(framewise_output: torch.Tensor, frames_num: int):
    """Pad framewise_output to the same length as input frames. The pad value
    is the same as the value of the last frame.
    Args:
      framewise_output: (batch_size, frames_num, classes_num)
      frames_num: int, number of frames to pad
    Outputs:
      output: (batch_size, frames_num, classes_num)
    """
    pad = framewise_output[:, -1:, :].repeat(
        1, frames_num - framewise_output.shape[1], 1)
    """tensor for padding"""

    output = torch.cat((framewise_output, pad), dim=1)
    """(batch_size, frames_num, classes_num)"""

    return output


class ConvBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.conv2 = nn.Conv2d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
            bias=False)

        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.init_weight()

    def init_weight(self):
        init_layer(self.conv1)
        init_layer(self.conv2)
        init_bn(self.bn1)
        init_bn(self.bn2)

    def forward(self, input, pool_size=(2, 2), pool_type='avg'):

        x = input
        x = F.relu_(self.bn1(self.conv1(x)))
        x = F.relu_(self.bn2(self.conv2(x)))
        if pool_type == 'max':
            x = F.max_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg':
            x = F.avg_pool2d(x, kernel_size=pool_size)
        elif pool_type == 'avg+max':
            x1 = F.avg_pool2d(x, kernel_size=pool_size)
            x2 = F.max_pool2d(x, kernel_size=pool_size)
            x = x1 + x2
        else:
            raise Exception('Incorrect argument!')

        return x


class AttBlock(nn.Module):
    def __init__(self, in_features: int, out_features: int, activation="linear", temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.bn_att = nn.BatchNorm1d(out_features)
        self.init_weights()

    def init_weights(self):
        init_layer(self.att)
        init_layer(self.cla)
        init_bn(self.bn_att)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        norm_att = torch.softmax(torch.clamp(self.att(x), -10, 10), dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)  

In [ ]:
encoder_params = {
    "tf_efficientnet_b0_ns": {
        "features": 1280,
        "init_op": partial(tf_efficientnet_b0_ns, pretrained=True, drop_path_rate=0.2)
    },
    "resnest50d": {
        "features": 2048,
        "init_op": partial(resnest50d, pretrained=True)
    }
}

class SEDModel(nn.Module):
    def __init__(self, encoder, sample_rate, window_size, hop_size, mel_bins, fmin, fmax, classes_num):
        super().__init__()

        amin = 1e-10
        center = True
        pad_mode = 'reflect'
        ref = 1.0
        self.interpolate_ratio = 30
        top_db = None
        window = 'hann'

        self.spectrogram_extractor = Spectrogram(n_fft=window_size, hop_length=hop_size, 
            win_length=window_size, window=window, center=center, pad_mode=pad_mode, freeze_parameters=True)
        
        self.logmel_extractor = LogmelFilterBank(sr=sample_rate, n_fft=window_size, 
            n_mels=mel_bins, fmin=fmin, fmax=fmax, ref=ref, amin=amin, top_db=top_db, freeze_parameters=True)

        self.spec_augmenter = SpecAugmentation(time_drop_width=64, time_stripes_num=2, freq_drop_width=8, freq_stripes_num=2)
        
        self.encoder = encoder_params[encoder]["init_op"]()
        self.fc = nn.Linear(encoder_params[encoder]["features"], 1024, bias=True)
        self.att_block = AttBlock(1024, classes_num, activation="sigmoid")
        self.bn = nn.BatchNorm2d(mel_bins)
        self.init_weight()
    
    def init_weight(self):
        init_layer(self.fc)
        init_bn(self.bn)
    
    def forward(self, input, mixup_lambda=None):
        x = self.spectrogram_extractor(input) # batch_size x 1 x time_steps x freq_bins
        x = self.logmel_extractor(x) # batch_size x 1 x time_steps x mel_bins

        frames_num = x.shape[2]

        x = x.transpose(1, 3)
        x = self.bn(x)
        x = x.transpose(1, 3)

        if self.training:
            x = self.spec_augmenter(x)
        
        if self.training and mixup_lambda is not None:
            x = do_mixup(x, mixup_lambda)
        
        x = x.expand(x.shape[0], 3, x.shape[2], x.shape[3]) # batch_size x 3 x time_steps x mel_bins
        x = self.encoder.forward_features(x)
        x = torch.mean(x, dim=3)

        x1 = F.max_pool1d(x, kernel_size=3, stride=1, padding=1)
        x2 = F.avg_pool1d(x, kernel_size=3, stride=1, padding=1)
        x = x1 + x2

        x = F.dropout(x, p=0.5, training=self.training)
        x = x.transpose(1, 2)
        x = F.relu_(self.fc(x))
        x = x.transpose(1, 2)
        x = F.dropout(x, p=0.5, training=self.training)

        (clipwise_output, norm_att, segmentwise_output) = self.att_block(x)
        logit = torch.sum(norm_att * self.att_block.cla(x), dim=2)
        segmentwise_output = segmentwise_output.transpose(1, 2)

        framewise_output = interpolate(segmentwise_output, self.interpolate_ratio)
        framewise_output = pad_framewise_output(framewise_output, frames_num)

        output_dict = {
            'framewise_output': framewise_output,
            'logit': logit,
            'clipwise_output': clipwise_output
        }

        return output_dict

### Define dataset

In [ ]:
def crop_or_pad(y, sr, period, record):
    sample_length = len(y)
    win_length = sr * period
    rint = np.random.randint(len(record['t_min']))
    time_start = record['t_min'][rint] * sr
    time_end = record['t_max'][rint] * sr
    if sample_length > win_length:
        c1 = time_end - win_length / 2
        c2 = time_start + win_length / 2
        if c1 < win_length / 2:
            c1 = win_length / 2
        if c2 > sample_length - win_length / 2:
            c2 = sample_length - win_length / 2
        center = np.random.randint(c1, c2)
        beginning = int(center - win_length / 2)
        ending = int(center + win_length / 2)
        y = y[beginning:ending].astype(np.float32)
    else:
        y = y.astype(np.float32)
        beginning = 0
        ending = win_length
    beginning_time = beginning / sr
    ending_time = ending / sr
    label = np.zeros(24, dtype='f')
    for i in range(len(record['t_min'])):
        if (record['t_min'][i] <= ending_time) and (record['t_max'][i] >= beginning_time):
            label[record['species_id'][i]] = 1
    return y, label

In [ ]:
class SEDDataset:
    
    def __init__(self, df, period=10, stride=10, audio_transform=None, data_path=None, mode=None):
        self.df = df.groupby("recording_id").agg(lambda x: list(x)).reset_index()
        if mode == 'train':
            self.df = self.df.append(self.df).reset_index(drop=True)
        self.period = period
        self.stride = stride
        self.audio_transform = audio_transform
        self.data_path = data_path
        self.mode = mode

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        record = self.df.iloc[idx]
        y, sr = sf.read(f"{self.data_path}/{record['recording_id']}.flac")
        
        if self.mode != "test":
            y, label = crop_or_pad(y, sr, period=self.period, record=record)
            if self.audio_transform:
                y = self.audio_transform(samples=y, sample_rate=sr)
        else:
            win_length = self.period * sr
            stride = self.stride * sr
            y = np.stack([y[i:i+win_length].astype(np.float32) for i in range(0, 60*sr+stride-win_length, stride)])
            label = np.zeros(24, dtype='f')
            
        return {
            "image": y,
            "target": label,
            "id": record['recording_id']
        }

### Augmentations

In [ ]:
train_audio_transform = aa.Compose([
    aa.AddGaussianNoise(p=0.5),
    aa.AddGaussianSNR(p=0.5),
    aa.Shift(p=0.5),
    aa.Gain(p=0.5),
    aa.AddBackgroundNoise(sounds_path="/kaggle/input/birdcall-noise-resample/")
])

### Utility functions

In [ ]:
def _lwlrap_sklearn(truth, scores):
    """Reference implementation from https://colab.research.google.com/drive/1AgPdhSp7ttY18O3fEoHOQKlt_3HJDLi8"""
    sample_weight = np.sum(truth > 0, axis=1)
    nonzero_weight_sample_indices = np.flatnonzero(sample_weight > 0)
    overall_lwlrap = metrics.label_ranking_average_precision_score(
        truth[nonzero_weight_sample_indices, :] > 0, 
        scores[nonzero_weight_sample_indices, :], 
        sample_weight=sample_weight[nonzero_weight_sample_indices])
    return overall_lwlrap

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class MetricMeter(object):
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.y_true = []
        self.y_pred = []
    
    def update(self, y_true, y_pred):
        self.y_true.extend(y_true.cpu().detach().numpy().tolist())
        self.y_pred.extend(y_pred.cpu().detach().numpy().tolist())

    @property
    def avg(self):
        self.score = _lwlrap_sklearn(np.array(self.y_true), np.array(self.y_pred))
        return {
            "lwlrap" : self.score
        }

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

### Losses

In [ ]:
class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()

        self.bce = nn.BCELoss()

    def forward(self, input, target):
        input_ = input["clipwise_output"]
        input_ = torch.where(torch.isnan(input_), torch.zeros_like(input_), input_)
        input_ = torch.where(torch.isinf(input_), torch.zeros_like(input_), input_)
        target = target.float()

        return self.bce(input_, target)

### Train function

In [ ]:
def train_epoch(args, model, loader, criterion, optimizer, scheduler, epoch):
    losses = AverageMeter()
    scores = MetricMeter()
    model.train()
    t = tqdm(loader)
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        input = sample['image'].to(args.device)
        target = sample['target'].to(args.device)
        M = Mixup(.4)
        lam = M.get_lambda(args.batch_size).to(args.device)
        target = do_mixup(target, lam)
        output = model(input, mixup_lambda=lam)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if scheduler and args.step_scheduler:
            scheduler.step()
        bs = input.size(0)
        scores.update(target, torch.sigmoid(torch.max(output['framewise_output'], dim=1)[0]))
        losses.update(loss.item(), bs)
        t.set_description(f"Train E: {epoch} - Loss: {losses.avg:0.4f}")
    t.close()
    return scores.avg, losses.avg
        
def valid_epoch(args, model, loader, criterion, epoch):
    losses = AverageMeter()
    scores = MetricMeter()
    model.eval()
    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            target = sample['target'].to(args.device)
            output = model(input)
            loss = criterion(output, target)
            bs = input.size(0)
            scores.update(target, torch.sigmoid(torch.max(output['framewise_output'], dim=1)[0]))
            losses.update(loss.item(), bs)
            t.set_description(f"Valid E: {epoch} - Loss: {losses.avg:0.4f}")
    t.close()
    return scores.avg, losses.avg

def test_epoch(args, model, loader):
    model.eval()
    pred_list = []
    id_list = []
    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):
            input = sample["image"].to(args.device)
            bs, seq, w = input.shape
            input = input.reshape(bs*seq, w)
            id = sample["id"]
            output = model(input)
            output = torch.sigmoid(torch.max(output['framewise_output'], dim=1)[0])
            output = output.reshape(bs, seq, -1)
            output = torch.sum(output, dim=1)
            output = output.cpu().detach().numpy().tolist()
            pred_list.extend(output)
            id_list.extend(id)
    return pred_list, id_list

### Main function

In [ ]:
def main(fold):
    
    seed_everything(args.seed)

    args.fold = fold
    
    args.save_path = os.path.join(args.output_dir, args.model_name)
    os.makedirs(args.save_path, exist_ok=True)

    train_df = pd.read_csv(args.train_csv)
    sub_df = pd.read_csv(args.sub_csv)
    if args.debug:
        train_df = train_df.sample(200)
    train_fold_df = train_df[train_df.kfold != fold]
    valid_fold_df = train_df[train_df.kfold == fold]
    
    train_dataset = SEDDataset(
        df=train_fold_df,
        audio_transform=train_audio_transform,
        data_path=args.train_data_path,
        mode="train")
    valid_dataset = SEDDataset(
        df=valid_fold_df,
        audio_transform=None,
        data_path=args.train_data_path,
        mode="valid")
    test_dataset = SEDDataset(
        df=sub_df,
        audio_transform=None,
        data_path=args.test_data_path,
        mode="test")
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=args.num_workers)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers)
    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=args.num_workers)

    model = SEDModel(**args.model_param)
    model = model.to(args.device)

    if args.pretrain_weights:
        print("Loading pretrained weights...")
        model.load_state_dict(torch.load(args.pretrain_weights, map_location=args.device), strict=False)
        model = model.to(args.device)
    else:
        criterion = PANNsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        num_train_steps = int(len(train_loader) * args.epochs)
        num_warmup_steps = int(0.1 * args.epochs * len(train_loader))
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)

        best_lwlrap = -np.inf
        early_stop_count = 0

        for epoch in range(args.start_epoch, args.epochs):
            train_avg, train_loss = train_epoch(args, model, train_loader, criterion, optimizer, scheduler, epoch)
            valid_avg, valid_loss = valid_epoch(args, model, valid_loader, criterion, epoch)

            if args.epoch_scheduler:
                scheduler.step()

            content = f"""
                      {time.ctime()} \n
                      Fold: {args.fold}, Epoch: {epoch}, lr: {optimizer.param_groups[0]['lr']:.7}\n
                      Train Loss: {train_loss:0.4f} - LWLRAP: {train_avg['lwlrap']:0.4f}\n
                      Valid Loss: {valid_loss:0.4f} - LWLRAP: {valid_avg['lwlrap']:0.4f}\n
                      """
            print(content)

            with open(f'{args.save_path}/log_{args.model_name}.txt', 'a') as appender:
                appender.write(content + '\n')

            if valid_avg['lwlrap'] > best_lwlrap:
                print(f"Validation LWLRAP improved from {best_lwlrap} to {valid_avg['lwlrap']}")
                torch.save(model.state_dict(), os.path.join(args.save_path, f'fold-{args.fold}.bin'))
                best_lwlrap = valid_avg['lwlrap']
                early_stop_count = 0
            else:
                early_stop_count += 1
            # torch.save(model.state_dict(), os.path.join(args.save_path, f'fold-{args.fold}_last.bin'))

            if args.early_stop == early_stop_count:
                print("Early stopping count reached: ", early_stop_count)
                break

        model.load_state_dict(torch.load(os.path.join(args.save_path, f'fold-{args.fold}.bin'), map_location=args.device))
        model = model.to(args.device)

    target_cols = sub_df.columns[1:].values.tolist()
    test_pred, ids = test_epoch(args, model, test_loader)
    print(np.array(test_pred).shape)

    test_pred_df = pd.DataFrame({
        "recording_id" : sub_df.recording_id.values
    })
    test_pred_df[target_cols] = test_pred
    test_pred_df.to_csv(os.path.join(args.save_path, f"fold-{args.fold}-submission.csv"), index=False)
    print(os.path.join(args.save_path, f"fold-{args.fold}-submission.csv"))
    

### Train folds

In [ ]:
main(fold=0)